# Exercise: Comparing Feature Selection Methods in a Machine Learning Pipeline

#### Objective

In this exercise, you'll build a machine learning pipeline using sklearn to compare different feature selection techniques:

- **Principal Component Analysis (PCA)**
- **Single Variable Classifier-based selection**
- **Backward feature selection**

You'll evaluate how these methods perform on the same dataset and understand why certain methods work better for specific types of data.

## Step 1: Import the Required Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector

## Step 2: Load the Dataset

In [2]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target

X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Step 3: Train-Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (398, 30)
Test set shape: (171, 30)


## Step 4: Feature Selection Method 1 - Principal Component Analysis (PCA)

PCA reduces the feature space by projecting the data onto fewer dimensions, while preserving variance.

In [4]:
pipeline_pca = Pipeline(
    [
        ("scaler", StandardScaler()),  # Feature scaling
        ("pca", PCA(n_components=5)),  # PCA with 5 components
        ("classifier", DecisionTreeClassifier(random_state=42)),  # Classifier
    ]
)

# Train the model
pipeline_pca.fit(X_train, y_train)

# Make predictions
y_pred_pca = pipeline_pca.predict(X_test)

# Evaluate the model
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f"Accuracy with PCA: {accuracy_pca:.4f}")

Accuracy with PCA: 0.9181


## Step 5: Feature Selection Method 2 - Tree Stump-Based Feature Selection

Single Variable Classifier-based selection involves fitting a weak learner (e.g., a shallow decision tree) and using the importance of features as a selection criterion.

In [5]:
pipeline_stump = Pipeline(
    [
        ("scaler", StandardScaler()),  # Feature scaling
        (
            "feature_selection",
            SelectFromModel(
                DecisionTreeClassifier(max_depth=1, random_state=42), threshold="mean"
            ),
        ),  # Tre stump-based feature selection
        ("classifier", DecisionTreeClassifier(random_state=42)),  # Classifier
    ]
)

# Train the model
pipeline_stump = pipeline_stump.fit(X_train, y_train)

# Make predictions
y_pred_single = pipeline_stump.predict(X_test)

# Evaluate the model
accuracy_stump = accuracy_score(y_test, y_pred_single)
print(f"Accuracy with Single Variable Classifier-based selection: {accuracy_stump:.4f}")

Accuracy with Single Variable Classifier-based selection: 0.8713


## Step 6: Feature Selection Method 3 - Backward Feature Selection Using

In backward selection, we start with all features and iteratively remove one feature at a time, training the model at each step. Complete the following code.

In [6]:
# Define the model (tree stump)
model = DecisionTreeClassifier(random_state=42)

# Define the SequentialFeatureSelector
sfs_backward = SequentialFeatureSelector(model, n_features_to_select=5, direction="backward")

# Fit the selector on the training data
sfs_backward.fit(X_train, y_train)

# Get the selected features
selected_features_backward = X_train.columns[sfs_backward.get_support()]

# Train the model with selected features
model.fit(X_train[selected_features_backward], y_train)

# Make predictions with selected features
y_pred_backward = model.predict(X_test[selected_features_backward])

# Evaluate the model
accuracy_backward = accuracy_score(y_test, y_pred_backward)
print(f"Selected features after backward selection: {selected_features_backward}")
print(f"Accuracy with backward selection: {accuracy_backward:.4f}")

Selected features after backward selection: Index(['mean texture', 'fractal dimension error', 'worst area',
       'worst compactness', 'worst concave points'],
      dtype='object')
Accuracy with backward selection: 0.9415


## Step 7: Comparison and Analysis

Why do you think one feature selection method performed better than the others? How does the nature of the dataset influence the effectiveness of each method?

In [7]:
print(f"Accuracy with PCA: {accuracy_pca:.4f}")
print(f"Accuracy with Single Variable Classifier-based selection: {accuracy_stump:.4f}")
print(f"Accuracy with backward selection: {accuracy_backward:.4f}")

Accuracy with PCA: 0.9181
Accuracy with Single Variable Classifier-based selection: 0.8713
Accuracy with backward selection: 0.9415
